##Libraries

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt

In [ ]:
import tensorflow
device_name = tensorflow.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
print("TensorFlow version:", tensorflow.__version__)

Found GPU at: /device:GPU:0
TensorFlow version: 2.8.0


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Apr 14 02:45:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    32W / 250W |    375MiB / 16280MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


## Functions

In [ ]:
# Create df of null_names, null_counts, null_proportion, and null_dtypes
def null_summary(df):
  var_null_list = df.columns[df.isna().any()]
  var_null_names = df[var_null_list].dtypes.index
  var_null_counts = df[var_null_list].isna().sum()
  var_null_proportion = df[var_null_list].isna().sum()/df[var_null_list].isna().count()
  var_null_dtypes = df[var_null_list].dtypes
  var_null_df = pd.DataFrame({'null_names' : var_null_names, 'null_counts' : var_null_counts, 'null_proportion' : var_null_proportion, 'null_dtypes' : var_null_dtypes})
  
  return var_null_df.reset_index(drop=True)

##Map

In [ ]:
fp = '/content/drive/MyDrive/Modules/Module_30_Capstone_4_Final_project/weather_data/'

In [ ]:
pfwc = pd.read_csv(fp + "pfwc.csv")
pfwc = pfwc.drop(columns = 'Unnamed: 0')

In [ ]:
locs = pd.read_csv(fp + "locs.csv")
locs = locs.rename(columns = {'Unnamed: 0':'id'})

In [ ]:
# Use tmin to do the location mapping
tmin = pd.read_csv(fp + "tmin.csv")
mplocs = tmin.groupby(['latitude', 'longitude']).count().reset_index()[['latitude', 'longitude']]
mplocs['id'] = np.arange(0, mplocs.shape[0])

In [ ]:
mplocs.to_csv(fp + "tmin_locs.csv")

In [ ]:
def quarters(df):

    minlat = df['latitude'].min()
    maxlat = df['latitude'].max()
    midlat = (maxlat + minlat)/2

    minlon = df['longitude'].min()
    maxlon = df['longitude'].max()
    midlon = (maxlon + minlon)/2

    q1 = df[(df['latitude'] > midlat) & (df['latitude'] <= maxlat) & (df['longitude'] >= minlon) & (df['longitude'] < midlon)]
    q2 = df[(df['latitude'] > midlat) & (df['latitude'] <= maxlat) & (df['longitude'] >= midlon) & (df['longitude'] <= maxlon)]

    q3 = df[(df['latitude'] >= minlat) & (df['latitude'] <= midlat) & (df['longitude'] >= minlon) & (df['longitude'] < midlon)]
    q4 = df[(df['latitude'] >= minlat) & (df['latitude'] <= midlat) & (df['longitude'] >= midlon) & (df['longitude'] <= maxlon)]

    return [q1, q2, q3, q4]

In [ ]:
locsl = quarters(locs)
mplocsl = quarters(mplocs)

In [ ]:
def dist(lat1, long1, lat2, long2):

    # Convert decimal degrees to radians 
    lat1, long1, lat2, long2 = map(radians, [lat1, long1, lat2, long2])
    # Haversine formula 
    dlon = long2 - long1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

In [ ]:
def find_nearest(dfm, lat, long):
    distances = dfm.apply(lambda row: dist(lat, long, row['latitude'], row['longitude']), axis=1)
    return dfm.loc[distances.idxmin(), 'id']

In [ ]:
for idx, jj in enumerate(locsl):
    print(idx)
    locsl[idx]['id2'] = locsl[idx].apply(lambda row: find_nearest(mplocsl[idx], row['latitude'], row['longitude']), axis=1)

newlocs = pd.concat(locsl, ignore_index=True)

0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


1
2
3


In [ ]:
newlocs.to_csv(fp + "newlocs.csv")

In [ ]:
mplocs = pd.read_csv(fp + "tmin_locs.csv")
newlocs = pd.read_csv(fp + "newlocs.csv")

In [ ]:
null_summary(newlocs)

,null_names,null_counts,null_proportion,null_dtypes


##Merge the mapped locations into the pfw df

In [ ]:
mplocs = mplocs.rename(columns={'id': 'id2'})
mplocs = mplocs.rename(columns={'latitude': 'newlat'})
mplocs = mplocs.rename(columns={'longitude': 'newlon'})

In [ ]:
wthr = newlocs.merge(mplocs, how='left', on=['id2'])

In [ ]:
pfwd = pfwc.merge(wthr[['latitude', 'longitude', 'newlat', 'newlon']], how='left', on=['latitude', 'longitude'])

In [ ]:
# Set the years to do the next merges in chunks by year
ystart = 2015
yend = 2021
years = list(np.arange(ystart, yend+1, 1))
years

[2015, 2016, 2017, 2018, 2019, 2020, 2021]

##Tmin

In [ ]:
tmin = tmin.rename(columns={'latitude': 'newlat'})
tmin = tmin.rename(columns={'longitude': 'newlon'})

In [ ]:
fn = 'pfw_tmin.csv'

In [ ]:
wthr = tmin
elem = 'tmin'
bb = pfwd

year = years[0]
wthryear = wthr[wthr['year'] == year]
byear = bb[bb['year'] == year]
merg = byear.merge(wthryear[['newlat', 'newlon', 'month', 'day', 'year', elem]],
                how='left', on=['newlat', 'newlon', 'month', 'day', 'year'])
  
for year in years[1:]:
    wthryear = wthr[wthr['year'] == year]
    byear = bb[bb['year'] == year]
    df = byear.merge(wthryear[['newlat', 'newlon', 'month', 'day', 'year', elem]],
                    how='left', on=['newlat', 'newlon', 'month', 'day', 'year'])
    merg = pd.concat([merg, df], ignore_index=True)

merg.to_csv(fp + fn)

In [ ]:
null_summary(merg)

,null_names,null_counts,null_proportion,null_dtypes
0,tmin,2,0.000007,float64


##Tmax

In [ ]:
pfwe = pd.read_csv(fp + fn)

In [ ]:
tmax = pd.read_csv(fp + "tmax.csv")

In [ ]:
tmax = tmax.rename(columns={'latitude': 'newlat'})
tmax = tmax.rename(columns={'longitude': 'newlon'})

In [ ]:
fn = 'pfw_tmin_tmax.csv'

In [ ]:
wthr = tmax
elem = 'tmax'
bb = pfwe

year = years[0]
wthryear = wthr[wthr['year'] == year]
byear = bb[bb['year'] == year]
merg = byear.merge(wthryear[['newlat', 'newlon', 'month', 'day', 'year', elem]],
                how='left', on=['newlat', 'newlon', 'month', 'day', 'year'])
  
for year in years[1:]:
    wthryear = wthr[wthr['year'] == year]
    byear = bb[bb['year'] == year]
    df = byear.merge(wthryear[['newlat', 'newlon', 'month', 'day', 'year', elem]],
                    how='left', on=['newlat', 'newlon', 'month', 'day', 'year'])
    merg = pd.concat([merg, df], ignore_index=True)

merg.to_csv(fp + fn)

In [ ]:
null_summary(merg)

,null_names,null_counts,null_proportion,null_dtypes
0,tmin,2,0.000007,float64
1,tmax,2,0.000007,float64


##Precipitation

In [ ]:
pfwf = pd.read_csv(fp + fn)

In [ ]:
pcpn = pd.read_csv(fp + "pcpn.csv")

In [ ]:
pcpn = pcpn.rename(columns={'latitude': 'newlat'})
pcpn = pcpn.rename(columns={'longitude': 'newlon'})

In [ ]:
fn = 'pfw_tmin_tmax_pcpn.csv'

In [ ]:
wthr = pcpn
elem = 'pcpn'
bb = pfwf

year = years[0]
wthryear = wthr[wthr['year'] == year]
byear = bb[bb['year'] == year]
merg = byear.merge(wthryear[['newlat', 'newlon', 'month', 'day', 'year', elem]],
                how='left', on=['newlat', 'newlon', 'month', 'day', 'year'])
  
for year in years[1:]:
    wthryear = wthr[wthr['year'] == year]
    byear = bb[bb['year'] == year]
    df = byear.merge(wthryear[['newlat', 'newlon', 'month', 'day', 'year', elem]],
                    how='left', on=['newlat', 'newlon', 'month', 'day', 'year'])
    merg = pd.concat([merg, df], ignore_index=True)

merg.to_csv(fp + fn)

In [ ]:
null_summary(merg)

,null_names,null_counts,null_proportion,null_dtypes
0,tmin,2,0.000007,float64
1,tmax,2,0.000007,float64
2,pcpn,2,0.000007,float64


##Merge with population and rural-urban data

In [ ]:
pfwg = pd.read_csv(fp + fn)

Remove the extra columns originating from the merges, and remove the pfw columns that are redundant with the newly aggregated population and rural-urban data.

In [ ]:
rem = ['newlat', 'newlon', 'Unnamed: 0', 'Unnamed: 0.1']
pfwg = pfwg.drop(columns = rem)

Now merge with the population and rural-urban data.

In [ ]:
popru = pd.read_csv("/content/drive/MyDrive/Modules/Module_30_Capstone_4_Final_project/urban_rural_pop/urban_rural_pop.csv")

In [ ]:
bird = popru.merge(pfwg[['sub_id', 'tmin', 'tmax', 'pcpn']], how='left', on=['sub_id'])
bird = bird.drop(columns = 'Unnamed: 0')

In [ ]:
null_summary(bird)

,null_names,null_counts,null_proportion,null_dtypes
0,tmin,2,0.000007,float64
1,tmax,2,0.000007,float64
2,pcpn,2,0.000007,float64


In [ ]:
bird = bird.dropna()

In [ ]:
bird.shape

(304510, 225)

In [ ]:
bird.head()

,sub_id,loc_id,latitude,longitude,state,month,day,year,proj_period_id,valid,...,numfeeders_other,count_area_size_sq_m_atleast,ruc_2013,uic_2013,pop_est,land_area,water_area,tmin,tmax,pcpn
0,S25821377,L356729,41.972319,-80.357978,PA,11,14,2015,PFW_2016,1,...,0.0,375.01,2,2,278452.0,2069.80,1966.00,33.0,43.0,0.33
1,S25821441,L1803877,40.042294,-75.734918,PA,11,14,2015,PFW_2016,1,...,0.0,375.01,1,1,515098.0,1943.81,22.60,40.0,56.0,0.00
2,S25822713,L4006197,41.891688,-73.900931,NY,11,14,2015,PFW_2016,1,...,0.0,1.01,1,1,294105.0,2060.67,76.97,36.0,52.0,0.01
3,S25822768,L1339929,38.782852,-77.198724,VA,11,14,2015,PFW_2016,1,...,0.0,1.01,1,1,1141040.0,1012.60,39.57,44.0,61.0,0.00
4,S25824425,L2410157,42.939565,-76.575397,NY,11,14,2015,PFW_2016,1,...,1.0,1.01,4,5,78323.0,1791.19,445.71,34.0,47.0,0.19


In [ ]:
bird.to_csv("/content/drive/MyDrive/Modules/Module_30_Capstone_4_Final_project/aggregated_data.csv")